In [1]:
!pip install llama-index
!pip install llama-index-llms-huggingface
!pip install llama-index-readers-web

!pip install llama-index-core
!pip install llama-index-llms-openai
!pip install llama-index-llms-replicate
!pip install llama-index-embeddings-huggingface


Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [2]:
import torch
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

/root/miniconda3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in Huggin

In [3]:
# Set prompt template for generation (optional)
from llama_index.core import PromptTemplate

def completion_to_prompt(completion):
   return f"<|im_start|>system\n<|im_end|>\n<|im_start|>user\n{completion}<|im_end|>\n<|im_start|>assistant\n"

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|im_start|>system\n{message.content}<|im_end|>\n"
        elif message.role == "user":
            prompt += f"<|im_start|>user\n{message.content}<|im_end|>\n"
        elif message.role == "assistant":
            prompt += f"<|im_start|>assistant\n{message.content}<|im_end|>\n"

    if not prompt.startswith("<|im_start|>system"):
        prompt = "<|im_start|>system\n" + prompt

    prompt = prompt + "<|im_start|>assistant\n"

    return prompt



In [4]:

# Set Qwen2 as the language model and set generation config
Settings.llm = HuggingFaceLLM(
    model_name="/root/autodl-tmp/model/qwen",
    tokenizer_name="/root/autodl-tmp/model/qwen",
    #model_name="Qwen/Qwen2-7B-Instruct",
    #tokenizer_name="Qwen/Qwen2-7B-Instruct",
    context_window=5000,
    max_new_tokens=2000,
    generate_kwargs={"temperature": 0.1, "top_k": 5, "top_p": 0.1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [5]:
# Set embedding model
Settings.embed_model = HuggingFaceEmbedding(
    model_name = "/root/autodl-tmp/model/Xorbits/bge-m3/"
)

In [6]:
# Set the size of the text chunk for retrieval
Settings.transformations = [SentenceSplitter(chunk_size=256)]


In [7]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=Settings.embed_model,
    transformations=Settings.transformations
)


In [8]:
query_engine = index.as_query_engine(similarity_top_k=3)
your_query = "我的大姐叫什么名字？"
response = query_engine.query(your_query)
print(response.response)

根据提供的上下文信息，你的大姐的名字是姜世清。


In [9]:
index.storage_context.persist()

In [10]:
# query_engine = index.as_query_engine(streaming=True)
# streaming_response = query_engine.query("我的父亲叫什么名字？")
# streaming_response.print_response_stream()

In [11]:
import pandas as pd
source_nodes = []
for item in response.source_nodes:
    node = item.node
    print(node.text)
#     score = item.score
#     title = node.metadata.get('file_name', None)
#     if title is None:
#         title = node.metadata.get('title', 'N/A') # if the document is a webpage, use the title
#         continue
#     page_label = node.metadata.get('page_label', 'N/A')
#     text = node.text
#     short_text = text[:50] + "..." if len(text) > 50 else text
#     source_nodes.append({"Title": title, "Page": page_label, "Text": short_text, "Score": f"{score:.2f}"})
# df = pd.DataFrame(source_nodes)
# df

我大姐——姜世清，在上世纪八十年代末（年份不详)病故（因为她去世后她的两个女儿并没有通知我们）。
姐弟妹四人先后离世，都是一次次地给父母沉重的打击，使他们悲痛欲绝。这真是：植树避阴，哪知枝残花落地。养儿防老，谁知父母送儿上青山！白发人送黑发人，多么悲惨和伤痛啊！
事不由人催人老！
”
我记忆里，大伯很少回家，更不知道大姐是他的女儿，都认为大姐是我父母亲生的，是我的亲姐姐。那时我还小，只觉得大姐高大，像母亲一样高，比母亲身体壮。会做事，对我和姐姐都很好，常教我们学做事，带着我们一起去割草、捡柴.
大姐从小跟着我父母长大，伯爷根本没管过，也未接大姐去他家耍过。大姐二十多岁结婚时的嫁妆、婚宴都是我父母操办的。
本来你祖父因为你大伯不懂事，也不喜欢这个孙女，后来见她长得乖巧，嘴也教得甜，转变了对她的态度，也经常抱抱她，牵着小手带她走耍，给她买糖果吃。”
我记忆里，大伯很少回家，更不知道大姐是他的女儿，都认为大姐是我父母亲生的，是我的亲姐姐。那时我还小，只觉得大姐高大，像母亲一样高，比母亲身体壮。会做事，对我和姐姐都很好，常教我们学做事，带着我们一起去割草、捡柴.
